<a href="https://colab.research.google.com/github/Madeira-International-Workshop-in-ML/2022_day_5/blob/main/Notebooks/Example%202%20--%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the necessary libraries

In [ ]:
import pathlib

import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files
from tqdm import tqdm

In [ ]:
# The following is just to check if the GPU from COLAB can is AVAILABLE
is_gpu_available = tf.config.list_physical_devices('GPU')
print('GPU is', 'AVAILABLE' if is_gpu_available else 'NOT AVAILABLE')

GPU is AVAILABLE


# Prepare the dataset

The MNIST database contains 60,000 training images and 10,000 testing images of handwritten digits. 

Each image in the MNIST dataset is a 28x28 grayscale image containing a digit from 0 to 9, and a label identifying which digit is in the image.
![MNIST sample](https://github.com/khanhlvg/DigitClassifier/raw/master/images/mnist.png)

In [ ]:
mnist = tf.keras.datasets.mnist  # The Fashion MNIST data is available directly in the tf.keras datasets API. On the
# first execution, the data will be downloaded. Note that then the data is cached

(x_train, y_train), (
        x_test, y_test) = mnist.load_data()  # Loads the MNIST dataset and returns the training and testing
# datasets. Note that we are going to use the training dataset
# for training the network, whereas the test dataset (which
# contains examples that were employed for training the network)
# will be used to assess the generalization capabilities of the
# network

x_train, x_test = x_train / 255.0, x_test / 255.0  # Let's normalize the dataset (both training and testing datasets)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# Add a color dimension to the images in "train" and "validate" dataset to
# leverage Keras's data augmentation utilities later.
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [ ]:
# Convert data to tf.float32
x_train = tf.dtypes.cast(x_train, tf.float32)
y_train = tf.dtypes.cast(y_train, tf.float32)
x_test = tf.dtypes.cast(x_test, tf.float32)
y_test = tf.dtypes.cast(y_test, tf.float32)

In [ ]:
# Define data augmentation
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.25,
        height_shift_range=0.25,
        shear_range=0.25,
        zoom_range=0.2
)

# Generate augmented data from MNIST dataset
train_generator = data_generator.flow(x_train, y_train)
test_generator = data_generator.flow(x_test, y_test)

# Create the TensorFlow model

We are going to use a simple convolutional neural network, which is a common technique in computer vision.

In [ ]:
# The following is to create the model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),  # turns the input into a 1 dimensional set
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # output layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Check to see if the model is what we pretend
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               204928    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [ ]:
# This is just for avoiding overfitting
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        if logs.get('accuracy') > 0.90:
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training = True

# Fit and test the model

In [ ]:
# Let's train the model
callbacks = MyCallback()
model.fit(train_generator, epochs=10, validation_data=test_generator, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 66s 20ms/step - loss: 0.6662 - accuracy: 0.7821 - val_loss: 0.3312 - val_accuracy: 0.8935
Epoch 2/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.2832 - accuracy: 0.9112 - val_loss: 0.2296 - val_accuracy: 0.9261

Reached 90% accuracy so cancelling training!


**Remember:** we are testing the model on images that the model has never seen before.

In [ ]:
# Evaluate the loss and accuracy from the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Loss {test_loss:.2f} and Accuracy {test_acc * 100:.2f}% on test dataset.')

313/313 [==============================] - 2s 5ms/step - loss: 0.0797 - accuracy: 0.9744
Loss 0.08 and Accuracy 97.44% on test dataset.


In [ ]:
# Evaluate the loss and accuracy from the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Loss {test_loss:.2f} and Accuracy {test_acc * 100:.2f}% on test dataset.')

313/313 [==============================] - 5s 16ms/step - loss: 0.2370 - accuracy: 0.9259
Loss 0.24 and Accuracy 92.59% on test dataset.


# Export the model

Now as we have trained the digit classifier model, we will convert it to TensorFlow Lite format for mobile deployment.

In [ ]:
# Export the SavedModel
export_dir = '/tmp/saved_model/1'
tf.saved_model.save(model, export_dir=export_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /tmp/saved_model/1/assets


## Perform post-training quantization

---

### Dynamic range quantization

The simplest form of post-training quantization statically quantizes **only the weights** from floating point to integer, which has 8-bits of precision.


### Full integer quantization

Makes sure that all model math is integer quantized. However, it uses float operators when they don't have an integer implementation.

For full integer quantization, you need to calibrate or estimate the range, i.e, (min, max) of all floating-point tensors in the model. Unlike constant tensors such as weights and biases, variable tensors such as model input, activations (outputs of intermediate layers) and model output cannot be calibrated unless we run a few inference cycles. As a result, the converter requires a representative dataset to calibrate them. This dataset can be a small subset (around ~100-500 samples) of the **training or validation** data. 


### Integer only

Ensure compatibility with integer only devices (such as 8-bit microcontrollers) and accelerators (such as the Coral Edge TPU) by enforcing full-integer quantization for all ops including the input and output.

**The converter will throw an error if it encounters an operation it cannot currently quantize.**



In [ ]:
type_quantization = 'Dynamic'  #@param ["none", "Dynamic", "Full", "Integer only"]


# For post-training quantization, let's define a representative dataset to calibrate variable tensors (e.g., model
# input, activations (outputs of intermediate layers). This dataset can be a small subset (around ~100--500 samples) of 
# the training or validation data
def representative_dataset():
    for _ in range(100):
        data = x_train[:100]
        yield [tf.dtypes.cast(data, tf.float32)]


# Prepare the converter with post-training quantization
if type_quantization == 'none':
    converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
    tflite_model = converter.convert()
elif type_quantization == 'Dynamic':
    converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
elif type_quantization == 'Full':
    converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    tflite_model = converter.convert()
elif type_quantization == 'Integer only':
    converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    tflite_model = converter.convert()

# Prepare path string
model_path = 'mnist.tflite'

# Save the model
tflite_model_file = pathlib.Path(model_path)
tflite_model_file.write_bytes(tflite_model)

251120

# TensorFlow Lite Interpreter

In [ ]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Gather results for the randomly sampled test images
predictions = []

# No. examples for testing
no_test = 2000  #@param {type:"integer"}

# Foreach testing sample
for i in tqdm(range(no_test)):

    # Extract the input
    if type_quantization == 'Integer only':
        img = np.expand_dims(x_test[i], axis=0).astype(np.int8)
    else:
        img = np.expand_dims(x_test[i], axis=0)
    interpreter.set_tensor(input_index, img)

    # Run the model
    interpreter.invoke()
    digit = np.argmax(interpreter.get_tensor(output_index)[0])

    # Collect the result
    predictions.append(digit)

100%|██████████| 2000/2000 [00:32<00:00, 62.18it/s]


In [ ]:
results = pd.DataFrame({'Real': y_test[:no_test], 'Predicted': predictions})
results['OK'] = results.Real == results.Predicted

print(f'Test accuracy {(np.sum(results["OK"] == True) / no_test) * 100.0:.5f}%.')

Test accuracy 96.70000%.


## Download the model for using on Android devices

In [ ]:
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>